https://python.langchain.com/docs/how_to/custom_tools/

In [3]:
from getpass import getpass

In [4]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

In [5]:
from ase.atoms import Atoms
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.constraints import UnitCellFilter
from ase.eos import calculate_eos, plot
from ase.data import reference_states, atomic_numbers
from ase.optimize import LBFGS
from ase.units import kJ

In [6]:
from langchain.agents import tool

In [7]:
import matplotlib.pyplot as plt

In [8]:
from langchain_core.pydantic_v1 import BaseModel

In [9]:
import numpy as np

In [10]:
from typing import List
from atomistics.calculators import evaluate_with_ase
from atomistics.workflows import EnergyVolumeCurveWorkflow, optimize_positions_and_volume
from ase.build import bulk



In [11]:
class AtomsDict(BaseModel):
    numbers: List[int]
    positions: List[List[float]]
    cell: List[List[float]]
    pbc: List[bool]

In [12]:
@tool 
def get_equilibirum_lattice(chemical_symbol: str) -> AtomsDict:
    """Returns equilibrium atoms dictionary for a given chemical symbol"""
    atoms = bulk(name=chemical_symbol)
    atoms.calc = EMT()
    ase_optimizer_obj = LBFGS(UnitCellFilter(atoms))
    ase_optimizer_obj.run(fmax=0.000001)
    return AtomsDict(**{k: v.tolist() for k, v in atoms.todict().items()})

In [13]:
@tool
def get_thermal_property(atom_dict: AtomsDict) -> dict:
    """Returns volumes at 100 and 1000 Kelvin, for a given atoms dictionary"""
    structure = Atoms(**atom_dict.dict())
    task_dict = optimize_positions_and_volume(structure=structure)
    result_dict = evaluate_with_ase(
    task_dict=task_dict,
    ase_calculator=EMT(),
    ase_optimizer=LBFGS,
    ase_optimizer_kwargs={"fmax": 0.000001}
                                    )
    
    workflow = EnergyVolumeCurveWorkflow(
    structure=result_dict["structure_with_optimized_positions_and_volume"],
    num_points=11,
    fit_type='polynomial',
    fit_order=3,
    vol_range=0.05,
    axes=('x', 'y', 'z'),
    strains=None,
                                 )
    task_dict = workflow.generate_structures()
    result_dict = evaluate_with_ase(task_dict=task_dict, ase_calculator=EMT())
    fit_dict = workflow.analyse_structures(output_dict=result_dict)
                                       
    thermal_properties_dict = workflow.get_thermal_properties(
    temperatures=[100, 1000],
    output_keys=["temperatures", "volumes"]
                                                            )
    return thermal_properties_dict

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [16]:
tools = [get_equilibirum_lattice, get_thermal_property]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [18]:
llm_with_tools = llm.bind_tools(tools)

In [19]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [20]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
lst = list(agent_executor.stream({"input": "Return th thermal property of gold"}))  # Yeah this worked !!



> Entering new AgentExecutor chain...

Invoking: `get_equilibirum_lattice` with `{'chemical_symbol': 'Au'}`


       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 18:56:59        0.002606*       0.3089
LBFGS:    1 18:56:59        0.000032*       0.0778
LBFGS:    2 18:56:59       -0.000135*       0.0031
LBFGS:    3 18:56:59       -0.000135*       0.0000
LBFGS:    4 18:56:59       -0.000135*       0.0000
numbers=[79] positions=[[0.0, 0.0, 0.0]] cell=[[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.0280828097056194, 3.558683810925054e-18]] pbc=[True, True, True]
Invoking: `get_thermal_property` with `{'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.02

In [24]:
lst[0]

{'actions': [ToolAgentAction(tool='get_equilibirum_lattice', tool_input={'chemical_symbol': 'Au'}, log="\nInvoking: `get_equilibirum_lattice` with `{'chemical_symbol': 'Au'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r', 'function': {'arguments': '{"chemical_symbol":"Au"}', 'name': 'get_equilibirum_lattice'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-bd42c2bb-b0e8-4aa3-acbd-1aab578177b5', tool_calls=[{'name': 'get_equilibirum_lattice', 'args': {'chemical_symbol': 'Au'}, 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r'}], tool_call_chunks=[{'name': 'get_equilibirum_lattice', 'args': '{"chemical_symbol":"Au"}', 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r', 'index': 0}])], tool_call_id='call_KuwqpdcORVA7xdqblaoAlM0r')],
 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r', 'function': {'arguments': '{"chemica

In [25]:
lst[1]

{'steps': [AgentStep(action=ToolAgentAction(tool='get_equilibirum_lattice', tool_input={'chemical_symbol': 'Au'}, log="\nInvoking: `get_equilibirum_lattice` with `{'chemical_symbol': 'Au'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r', 'function': {'arguments': '{"chemical_symbol":"Au"}', 'name': 'get_equilibirum_lattice'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-bd42c2bb-b0e8-4aa3-acbd-1aab578177b5', tool_calls=[{'name': 'get_equilibirum_lattice', 'args': {'chemical_symbol': 'Au'}, 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r'}], tool_call_chunks=[{'name': 'get_equilibirum_lattice', 'args': '{"chemical_symbol":"Au"}', 'id': 'call_KuwqpdcORVA7xdqblaoAlM0r', 'index': 0}])], tool_call_id='call_KuwqpdcORVA7xdqblaoAlM0r'), observation=AtomsDict(numbers=[79], positions=[[0.0, 0.0, 0.0]], cell=[[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.028082809

In [26]:
lst[2]

{'actions': [ToolAgentAction(tool='get_thermal_property', tool_input={'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.0280828097056194, 3.558683810925054e-18]], 'pbc': [True, True, True]}}, log="\nInvoking: `get_thermal_property` with `{'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.0280828097056194, 3.558683810925054e-18]], 'pbc': [True, True, True]}}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UEUtZQOLQacbJlwcBjaz2Y7H', 'function': {'arguments': '{"atom_dict":{"numbers":[79],"positions":[[0.0,0.0,0.0]],"cell":[[4.6804645871046276e-17,2.0280828097056194,2.0280828097056203

In [27]:
lst[3]

{'steps': [AgentStep(action=ToolAgentAction(tool='get_thermal_property', tool_input={'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.0280828097056194, 3.558683810925054e-18]], 'pbc': [True, True, True]}}, log="\nInvoking: `get_thermal_property` with `{'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[4.6804645871046276e-17, 2.0280828097056194, 2.0280828097056203], [2.0280828097056185, 9.184893482322169e-17, 2.0280828097056203], [2.0280828097056185, 2.0280828097056194, 3.558683810925054e-18]], 'pbc': [True, True, True]}}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UEUtZQOLQacbJlwcBjaz2Y7H', 'function': {'arguments': '{"atom_dict":{"numbers":[79],"positions":[[0.0,0.0,0.0]],"cell":[[4.6804645871046276e-17,2.0280828097056194,2.0

In [28]:
lst[4]

{'output': 'The thermal property of gold at 100 Kelvin has a volume of 16.75 and at 1000 Kelvin has a volume of 17.24.',
 'messages': [AIMessage(content='The thermal property of gold at 100 Kelvin has a volume of 16.75 and at 1000 Kelvin has a volume of 17.24.')]}

# Lessons learned: 
* Using Atomistics to calculate property works well
* need to try with other properties"